In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
#import matplotlib.pyplot as plt
import numpy as np

import os
import shutil
#import pandas
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.metrics import mean_squared_error
import math


import tensorflow as tf

from tensorflow.keras import layers
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)



In [3]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()


In [86]:
def insertarPalabra(palabra, palabras, posiciones, minuscula, palabrasInvalidas=["\n"]):
    if len(palabra)==0 or palabra in palabrasInvalidas:
        return
    if minuscula:
        palabra=palabra.lower()
    if(palabra not in palabras):
        palabras.append(palabra)
    indice=palabras.index(palabra)
    posiciones.append(indice)
#Vocabulario obtiene tanto la lista de palabras como las posiciones por párrafo de cada palabra en el texto dado
#juntarParrafos tiene en cuenta que un \n que no está precedido por un "." o un ". " no se considera como parrafo nuevo
def vocabulario(fichero, caracterSeparacion=' ', caracteresPuntuacion=['!','"','#','$','%','&','(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\','\'',']','^','_','`','{','|','}','~','\t','\n','\r','¿','¡'], caracteresDescartados=['\r', '\t'], minuscula=True, palabras=[], conParrafos=False, juntarParrafos=False, caracteresFinalesParrafo=['.','_']):
    posiciones=[]
    if not isinstance(fichero,list):
        ficheros=[fichero]
    else:
        ficheros=fichero
    for fichero in ficheros:
        with open(fichero,mode="r",encoding="utf8") as fichero:
            ultimoCaracter=""
            palabra=""
            esPuntuacion=False
            palabrasInvalidas=caracteresDescartados
            if conParrafos:
                parrafo=[]
                palabrasInvalidas=["\n"]
            else:
                parrafo=posiciones
            for linea in fichero:
                for c in linea:
                    
                    if(c==caracterSeparacion):
                        if(esPuntuacion):
                            palabra+=c
                        insertarPalabra(palabra, palabras, parrafo, minuscula,palabrasInvalidas)
                        palabra=""
                        esPuntuacion=False
                    else:
                        if(c not in caracteresPuntuacion):
                            if(esPuntuacion):
                                insertarPalabra(palabra, palabras, parrafo,minuscula,palabrasInvalidas)
                                esPuntuacion=False
                                palabra=c
                            else:
                                palabra+=c
                        else:
                            insertarPalabra(palabra, palabras, parrafo,minuscula,palabrasInvalidas)
                            palabra=c
                            esPuntuacion=True
                    if c!=caracterSeparacion and c not in caracteresDescartados and c!='\n':
                        ultimoCaracter=c
                
                if conParrafos and len(parrafo)>0:
                    if(not juntarParrafos or ultimoCaracter in caracteresFinalesParrafo):
                        posiciones.append(parrafo)
                        parrafo=[]
        if(len(parrafo)>0):
            posiciones.append(parrafo)
            parrafo=[]
    return palabras,posiciones

In [107]:
def dividir(posiciones,longitud_bloque):
    conjunto=[]
    elemento=0
    bloque=[]
    for i,posicion in enumerate(posiciones):
        if elemento==10:
            if(len(bloque)>0):
                conjunto.append(bloque)
                bloque=[]
                elemento=0
        bloque.append(posicion)
        elemento+=1
    if(len(bloque)>0):
        longitud=len(bloque)
        for i in range(longitud_bloque-longitud):
            bloque.append(0)
        conjunto.append(bloque)
    return conjunto

def dividir2(posiciones,longitud_bloque):
    conjunto=[]
    elemento=0
    for i in range(len(posiciones)-longitud_bloque):
        conjunto.append(np.array(posiciones[i:i+longitud_bloque]))
    return conjunto

def normalizar(datos):
    longitud_maxima=0
    for bloque in datos:
        if(len(bloque)>longitud_maxima):
            longitud_maxima=len(bloque)
    for bloque in datos:
        if len(bloque)<longitud_maxima:
            bloque.extend([0] * (longitud_maxima-len(bloque)))
    return datos,longitud_maxima

def agrupar(posiciones,longitud_bloque, longitud_maxima):
    #Generamos agrupaciones con bloques del mismo tamaño
    conjunto=[]
    #Calculamos el tamaño del bloque más grande
    

    elemento=0
    bloque=[]
    for i,posicion in enumerate(posiciones):
        if len(posicion)<longitud_maxima:
            posicion.extend([0] * (longitud_maxima-len(posicion)))
        if elemento==10:
            if(len(bloque)>0):
                conjunto.append(bloque)
                bloque=[]
                elemento=0
        bloque.append(posicion)
        elemento+=1
    if(len(bloque)>0):
        longitud=len(bloque)
        for i in range(longitud_bloque-longitud):
            bloque.extend([0]*longitud_maxima)
        conjunto.append(bloque)
    return conjunto

In [113]:
#Devuelve un texto completo
def datosEntrenamiento1():
    palabras,posiciones=vocabulario("textos\libro.txt", palabras=["\n"])

    #numeroEntrenamiento=math.floor(len(posiciones)*0.8)
    #entrenamiento=posiciones[:numeroEntrenamiento]
    #test=posiciones[numeroEntrenamiento:]

    longitud_bloque=10
    etiquetas=[]
    entrenamiento=dividir2(posiciones,longitud_bloque)
    for i in range(len(entrenamiento)):
        etiquetas.append(0)
    entrenamiento=np.array(entrenamiento)
    etiquetas=np.array(etiquetas)
    return palabras, posiciones, entrenamiento,etiquetas

#Obtiene el texto en parrafos
def datosEntrenamiento2():
    palabras,posiciones=vocabulario(["textos\libro.txt","textos\libro2.txt","textos\libro3.txt"], palabras=["\n"],juntarParrafos=True,conParrafos=True)

    numeroEntrenamiento=math.floor(len(posiciones)*0.8)
    #entrenamiento=posiciones[:numeroEntrenamiento]
    #test=posiciones[numeroEntrenamiento:]

    longitud_bloque=10
    etiquetas=[]
    #entrenamiento=agrupar(posiciones,longitud_bloque)

    entrenamiento=[]
    posicionse,l=normalizar(posiciones)
    for i in range(len(posiciones)):
        etiquetas.append(0)
        entrenamiento.append(np.array(posiciones[i]))
        proporcion=np.random.randint(100)
        if(proporcion<20):
            etiquetas.append(1)
            entrenamiento.append(np.random.shuffle(np.array(posiciones[i])))
    entrenamiento=np.array(entrenamiento)
    entrenamiento=np.reshape(entrenamiento, (entrenamiento.shape[0], l))
    etiquetas=np.array(etiquetas)
    return palabras, posiciones, entrenamiento,etiquetas
#test=dividir(test,longitud_bloque)

palabras,posiciones,entrenamiento,etiquetas=datosEntrenamiento2()

print("Vocabulario=",len(palabras))
print("Entrenamiento=",entrenamiento.shape)
print("Etiquetas=",etiquetas.shape)
#print("test=",len(test))
entrenamiento[0]


AttributeError: 'list' object has no attribute 'shape'

In [105]:
#embedding_layer = layers.Embedding(len(posiciones), 2)
#result = embedding_layer(tf.constant(posiciones))
#result.numpy()


array([[-0.00512495,  0.04321748],
       [ 0.03834225, -0.01898133],
       [ 0.00431837, -0.03773835],
       ...,
       [-0.03060499,  0.01168156],
       [ 0.0071156 , -0.02809701],
       [-0.0082266 ,  0.03553892]], dtype=float32)

In [65]:
def crearModelo(longitud_vocabulario,longitud_embedding=16, tamañoBloque=10 ):
    model = tf.keras.Sequential([
    layers.Embedding(longitud_vocabulario, longitud_embedding),
    layers.GlobalAveragePooling1D(),
    layers.Dense(longitud_embedding, activation='relu'),
    layers.Dense(1)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model


In [109]:
modelo=crearModelo(len(palabras), tamañoBloque=longitud_bloque)
history = modelo.fit(
    x=entrenamiento, y=etiquetas,
    epochs=10,
    validation_split=0.2, validation_steps=20)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [25]:
import matplotlib.pyplot as plt

history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss=history_dict['loss']
val_loss=history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()


<Figure size 1200x900 with 1 Axes>

<Figure size 1200x900 with 1 Axes>

In [83]:

e = modelo.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


(6715, 16)


In [85]:
import io



vectores = io.open('modelos/vecs.tsv', 'w', encoding='utf-8')
metadatos = io.open('modelos/meta.tsv', 'w', encoding='utf-8')

for i, palabra in enumerate(palabras):
  vec = weights[i]
  metadatos.write(palabra + "\n")
  vectores.write('\t'.join([str(x) for x in vec]) + "\n")
vectores.close()
metadatos.close()


In [16]:
model = tf.keras.Sequential()
model.add(layers.Embedding(1000, 64, input_length=10))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be
# no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.

input_array = np.random.randint(1000, size=(32, 10))
#print(input_array)
model.compile('rmsprop', 'mse')
history = modelo.fit(
    input_array,
    epochs=10,
    validation_split=0.2, validation_steps=20)
#output_array = model.predict(input_array)
#assert output_array.shape == (32, 10, 64)

Train on 25 samples, validate on 7 samples
Epoch 1/10
25/25 [==============================] - 0s 720us/sample


IndexError: list index out of range